# UFC Fight Prediction

1) Imports and Initialize Spark Session

In [1]:
#Import neccesary libraries
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, col, when, count, expr
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import regexp_extract, col, expr
from pyspark.sql.functions import regexp_replace
from pyspark.sql import functions as F

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
np.bool = np.bool_
#current notebook name
notebook_name = __session__.replace('.ipynb','')[__session__.rfind('/')+1:] 
# HDFS base paths
hdfs_lakehouse_base_path = 'hdfs://localhost:9000/lakehouse/'
hdfs_warehouse_base_path = 'hdfs://localhost:9000/warehouse'

In [3]:
import os
dependencies = ["org.apache.spark:spark-avro_2.12:3.5.0",
                "io.delta:delta-iceberg_2.12:3.0.0"]
os.environ['PYSPARK_SUBMIT_ARGS']= f"--packages {','.join(dependencies)} pyspark-shell"
os.environ['PYARROW_IGNORE_TIMEZONE'] = 'true'

In [4]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName(notebook_name)
    .config("spark.log.level","ERROR")
    .config("spark.sql.warehouse.dir",hdfs_warehouse_base_path)
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .enableHiveSupport()
    .getOrCreate()
)

24/12/16 01:44:32 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s1)
24/12/16 01:44:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-iceberg_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11fe4553-621e-4d83-9a22-3ce1d9a0bace;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-iceberg_2.12;3.0.0 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.1.1 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.3 in central
	found org.checkerframework#checker-qual;3.19.0 in central
	found com.google.errorprone#error_prone_annotations;2.10.0 in central
:: resolution report :: resolve 247ms :: artifacts dl 17ms
	:: modules in use:
	com.github.ben-ma

2. Load Datasets

In [5]:
fighters_df = spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fighters")

In [6]:
fighters_df.toPandas().head()

,url,fighter_name,nickname,birth_date,age,location,country,height,weight,association,weight_class,wins,wins_ko,wins_submission,wins_decision,lossess,losses_ko,losses_submission,losses_decision
0,/fighter/Kharun-Atlangeriev-167431,Kharun Atlangeriev,Predator,05/10/1992,28,Unknown,Russia,70,155,OFS Team,Unknown,11,5,5,1,11,0,1,1
1,/fighter/Ryan-Keenan-33827,Ryan Keenan,Who Dat?,15/06/1985,36,"Windermere, Florida",United States,72,155,Gracie Orlando,Unknown,6,3,3,0,6,1,1,0
2,/fighter/Dustin-Jacoby-73825,Dustin Jacoby,The Hanyak,04/04/1988,33,"Arenzville, Illinois",United States,76,205,Factory X,Unknown,14,9,1,4,14,1,2,2
3,/fighter/Richardson-Moreira-72825,Richardson Moreira,Rick Monstro,31/03/1984,37,"Campinas, Sao Paulo",Brazil,74,205,Team Nogueira,Unknown,8,2,5,1,8,1,1,3
4,/fighter/Chris-Haseman-1449,Chris Haseman,The Hammer,02/06/1969,52,Unknown,Australia,71,198,Rings Australia,Unknown,20,2,14,3,20,8,3,6


In [7]:
upcoming_df=spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fights/upcoming_fights")

In [8]:
upcoming_df.toPandas().head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,AgeDif,SigStrDif,AvgSubAttDif,AvgTDDif,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Alexandre Pantoja,Kai Asakura,-250,215,40.0000,215.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,-3,-4.41,-0.8,-2.61,300,800,150,2500,400,350
1,Shavkat Rakhmonov,Ian Machado Garry,-210,295,47.6190,295.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,-3,1.38,-1.5,-0.72,250,650,180,3000,240,700
2,Ciryl Gane,Alexander Volkov,-380,300,26.3158,300.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,2,-0.36,-0.3,-0.13,-160,450,1100,3000,350,1100
3,Bryce Mitchell,Kron Gracie,-950,625,10.5263,625.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,6,1.44,-1.1,-2.98,-200,1100,380,1400,500,4000
4,Nate Landwehr,Dooho Choi,-130,110,76.9231,110.0,2024-12-07,"Las Vegas, Nevada, USA",USA,None,...,-3,-1.84,-0.2,-0.25,275,550,500,700,300,250


In [9]:
history_df=spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-fights/previous_fights")

In [10]:
history_df.toPandas().head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,Date,Location,Country,Winner,TitleBout,WeightClass,...,FinishDetails,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds
0,Petr Yan,Deiveson Figueiredo,-305.0,245.0,2024-11-23,"Macau, China",China,Red,False,Bantamweight,...,N/A,5.0,2024-12-15 05:00:00,1500.0,110.0,450.0,2000.0,1600.0,200.0,1000.0
1,Yan Xiaonan,Tabatha Ricci,-198.0,164.0,2024-11-23,"Macau, China",China,Red,False,Women's Strawweight,...,N/A,3.0,2024-12-15 05:00:00,900.0,110.0,215.0,2000.0,1100.0,200.0,1800.0
2,Song Kenan,Muslim Salikhov,-150.0,-230.0,2024-11-23,"Macau, China",China,Blue,False,Welterweight,...,Kick,1.0,2024-12-15 03:49:00,229.0,250.0,175.0,750.0,1800.0,420.0,200.0
3,Wang Cong,Gabriella Fernandes,-950.0,750.0,2024-11-23,"Macau, China",China,Blue,False,Women's Flyweight,...,Rear Naked Choke,2.0,2024-12-15 03:49:00,529.0,-110.0,1000.0,460.0,3000.0,280.0,2200.0
4,Volkan Oezdemir,Carlos Ulberg,215.0,-265.0,2024-11-23,"Macau, China",China,Blue,False,Light Heavyweight,...,N/A,3.0,2024-12-15 05:00:00,900.0,500.0,240.0,1600.0,800.0,500.0,130.0


In [11]:
rankings_df=spark.read.format("delta").load(f"{hdfs_lakehouse_base_path}/silver/UFC/UFC-rankings")

In [12]:
rankings_df.toPandas().head()

,date,weightclass,fighter,rank
0,2013-02-04,Featherweight,Chad Mendes,1
1,2013-02-11,Bantamweight,Renan Barao,1
2,2013-04-08,Bantamweight,Raphael Assuncao,6
3,2013-04-08,Middleweight,Vitor Belfort,2
4,2013-04-29,Pound-for-Pound,Frankie Edgar,10


3) Dataframe transformation

Fighter Performance Metrics

In [13]:
# Win/Loss Ratio for Red Fighter
history_df_clean = history_df.withColumn(
    "RedWinLossRatio", when(col("RedWins").isNotNull(), col("RedWins") / (col("RedLosses") + 1)).otherwise(0)
)

# Win/Loss Ratio for Blue Fighter
history_df_clean = history_df.withColumn(
    "BlueWinLossRatio", when(col("BlueWins").isNotNull(), col("BlueWins") / (col("BlueLosses") + 1)).otherwise(0)
)

In [14]:
history_df_clean.show(5)

+---------------+-------------------+-------+--------+----------+------------+-------+------+---------+-------------------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+------------------

Physical Differences

In [15]:
# Weight Difference
history_df_clean = history_df_clean.withColumn("WeightDiff", col("RedWeightLbs") - col("BlueWeightLbs"))

In [16]:
history_df_clean.show(5)
#Most have no weight difference since they are in the same class but some do (missed weight)

+---------------+-------------------+-------+--------+----------+------------+-------+------+---------+-------------------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+------------------

In [17]:
# Age Difference
history_df_clean = history_df_clean.withColumn("AgeDiff", col("RedAge") - col("BlueAge"))

In [18]:
history_df_clean.show(5)

+---------------+-------------------+-------+--------+----------+------------+-------+------+---------+-------------------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+------------------

Fight Statistics

In [19]:
# Significant Strike Ratio
history_df_clean = history_df_clean.withColumn(
    "SigStrRatio", (col("RedAvgSigStrLanded") + 1) / (col("BlueAvgSigStrLanded") + 1)
)

# Takedown Ratio
history_df_clean = history_df_clean.withColumn(
    "TDLandedRatio", (col("RedAvgTDLanded") + 1) / (col("BlueAvgTDLanded") + 1)
)

Fight Experience

In [20]:
# Total Experience for Red Fighter
history_df_clean = history_df_clean.withColumn("RedTotalFights", col("RedWins") + col("RedLosses") + col("RedDraws"))

# Total Experience for Blue Fighter
history_df_clean = history_df_clean.withColumn("BlueTotalFights", col("BlueWins") + col("BlueLosses") + col("BlueDraws"))

In [21]:
# Decision Odds Difference
history_df_clean = history_df_clean.withColumn("DecOddsDiff", col("RedDecOdds") - col("BlueDecOdds"))

# Submission Odds Difference
history_df_clean = history_df_clean.withColumn("SubOddsDiff", col("RSubOdds") - col("BSubOdds"))

# KO/TKO Odds Difference
history_df_clean = history_df_clean.withColumn("KOOddsDiff", col("RKOOdds") - col("BKOOdds"))

In [22]:
history_df_clean.show(5)

+---------------+-------------------+-------+--------+----------+------------+-------+------+---------+-------------------+------+--------------+---------------------+--------------------+---------+-------------------+----------------+-------------+---------------+------------+--------------------+----------+---------------------+-------------------+--------------------------+-----------------------+---------------------------+------------+--------------------+---------------------------+--------+----------+-------------+------------+-------------+--------------------+-------------------+--------+------------------+---------------+------------+--------------+-----------+-------------------+---------+--------------------+------------------+-------------------------+----------------------+--------------------------+-----------+-------------------+--------------------------+-------+---------+------------+-----------+------------+------+-------+-------------+------------+------------------

Categorical Encodings

String Indexing - High Cardinality Features

In [23]:
country_indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex")
history_df_clean = country_indexer.fit(history_df_clean).transform(history_df_clean)

In [24]:
history_df_clean.select("CountryIndex", "Country").show(5)

+------------+-------+
|CountryIndex|Country|
+------------+-------+
|        31.0|  China|
|        31.0|  China|
|        31.0|  China|
|        31.0|  China|
|        31.0|  China|
+------------+-------+
only showing top 5 rows



Stance Encoding

In [25]:
# Encode Red Fighter's Stance
history_df_clean = history_df_clean.withColumn(
    "RedStanceEncoded",
    when(col("RedStance") == "Orthodox", 1)
    .when(col("RedStance") == "Southpaw", 2)
    .otherwise(0)
)

# Encode Blue Fighter's Stance
history_df_clean = history_df_clean.withColumn(
    "BlueStanceEncoded",
    when(col("BlueStance") == "Orthodox", 1)
    .when(col("BlueStance") == "Southpaw", 2)
    .otherwise(0)
)

history_df_clean.select("RedStance", "RedStanceEncoded", "BlueStance", "BlueStanceEncoded").show(5)


+---------+----------------+----------+-----------------+
|RedStance|RedStanceEncoded|BlueStance|BlueStanceEncoded|
+---------+----------------+----------+-----------------+
|   Switch|               0|  Orthodox|                1|
| Orthodox|               1|  Orthodox|                1|
| Orthodox|               1|  Orthodox|                1|
| Southpaw|               2|  Southpaw|                2|
| Orthodox|               1|  Orthodox|                1|
+---------+----------------+----------+-----------------+
only showing top 5 rows



In [33]:
#(fighters_df_clean.write
#            .format("delta")
#            .mode("overwrite")
#            .option("path",f"{hdfs_lakehouse_base_path}/gold/UFC/UFC-fighters")
#            .saveAsTable("ufc.fighters")
#)

#(history_df_clean.write
#            .format("delta")
#            .mode("overwrite")
#            .option("path",f"{hdfs_lakehouse_base_path}/gold/UFC/UFC-fights/previous_fights")
#            .saveAsTable("ufc.previous_fights")
#)

#(upcoming_df_clean.write
#            .format("delta")
#            .mode("overwrite")
#            .option("path",f"{hdfs_lakehouse_base_path}/gold/UFC/UFC-fights/upcoming_fights")
#            .saveAsTable("ufc.upcoming_fights")
#)

#(rankings_df_clean.write
#            .format("delta")
#            .mode("overwrite")
#            .option("path",f"{hdfs_lakehouse_base_path}/gold/UFC/UFC-rankings")
#            .saveAsTable("ufc.rankings")
#)